In [ ]:
!pip install --upgrade scikit-learn==0.24.2

In [1]:
import sklearn

print(f"Scikit Learn Version: {sklearn.__version__}")

Scikit Learn Version: 0.24.2


In [2]:
import pandas as pd

#### Toy datasets

Choisir un autre toy dataset parmi ceux présentés sur cette page et adapter le code

https://scikit-learn.org/stable/datasets/toy_dataset.html

In [3]:
from sklearn.datasets import load_wine

dataset = sklearn.datasets.load_wine() #as_frame=True

In [4]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)

In [5]:
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [6]:
df.iloc[:,1]

0      1.71
1      1.78
2      2.36
3      1.95
4      2.59
       ... 
173    5.65
174    3.91
175    4.28
176    2.59
177    4.10
Name: malic_acid, Length: 178, dtype: float64

#### Train test split

In [8]:
# Les objets ci-dessous sont des numpy.ndarray
X = dataset.data
y = dataset.target

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Training

Choisir un autre algorithme du framework Scikit Learn et adapter le code

https://scikit-learn.org/stable/

In [10]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 5)
model.fit(X_train, y_train)   

KNeighborsClassifier()

In [11]:
y_pred = model.predict(X_test)

#### Hyperparameter tuning

Adapter les hyperparamètres en fonction de l'algorithme choisi

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {"n_neighbors": [2,3,4,5]}
gs_model = GridSearchCV(KNeighborsClassifier(), parameters)
gs_model.fit(X_train, y_train) 

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 3, 4, 5]})

In [13]:
print(gs_model.best_params_)

{'n_neighbors': 5}


In [14]:
y_pred = gs_model.predict(X_test)

#### Evaluation

In [15]:
print("Train accuracy : {}".format(model.score(X_train, y_train)))
print("Test accuracy : {}".format(model.score(X_test, y_pred)))

Train accuracy : 0.7887323943661971
Test accuracy : 1.0


#### Artefact

In [16]:
import os

os.makedirs('models', exist_ok=True)

In [25]:
model_name = 'wine_knn_model'
model_path = 'models/' + model_name + '.pkl'

In [26]:
import joblib

joblib.dump(model, model_path)

['models/wine_knn_model.pkl']

In [27]:
unpkl_model = joblib.load(model_path)

In [28]:
y_pred = unpkl_model.predict(X_test)
y_pred

array([1, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 0, 2,
       0, 1, 2, 1, 2, 0, 1, 1, 2, 2, 1, 0, 0, 0])

In [29]:
gs_model_name = 'wine_knn_gs_model'
gs_model_path = 'models/' + gs_model_name + '.pkl'

In [30]:
import joblib

joblib.dump(gs_model, gs_model_path)

['models/wine_knn_gs_model.pkl']

### Register model

In [31]:
from azureml.core import Workspace


ws = Workspace.from_config()
ws

Workspace.create(name='wshandsonaml', subscription_id='23376daf-77f3-4195-9649-e223b8072ad9', resource_group='rg-handsonaml')

In [32]:
from azureml.core.model import Model


aml_model = Model.register(workspace=ws,
                       model_name=model_name,
                       model_path=model_path,
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,             # Version of scikit-learn used to create the model.
                       tags={'dataset': "wine (toy dataset)", 'type': "KNN classification"},
                       description="KNN classification for wine class"
                       )

Registering model wine_knn_model


In [33]:
aml_model.version

4

In [34]:
aml_gs_model = Model.register(workspace=ws,
                       model_name=gs_model_name,
                       model_path=gs_model_path,
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,             # Version of scikit-learn used to create the model.
                       tags={'dataset': "wine (toy dataset)", 'type': "KNN classification"},
                       description="KNN classification with Grid Search for wine class"
                       )

Registering model wine_knn_gs_model


Choisir dans la cellule ci-dessous le modèle qui sera déployé ainsi que sa version

In [36]:
from azureml.core.model import Model

model = Model(ws, 'wine_knn_gs_model', version=1)

### Deploy service

In [37]:
# Use a default environment
# nécessite d'avoir précisé le framework lors de l'enregistrement du modèle
# Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow']."

from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'wine-default-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-16 14:11:24+00:00 Creating Container Registry if not exists.
2021-06-16 14:11:24+00:00 Registering the environment.
2021-06-16 14:11:25+00:00 Uploading autogenerated assets for no-code-deployment.
2021-06-16 14:11:25+00:00 Use the existing image.
2021-06-16 14:11:26+00:00 Generating deployment configuration.
2021-06-16 14:11:27+00:00 Submitting deployment to compute..
2021-06-16 14:11:35+00:00 Checking the status of deployment wine-default-service..
2021-06-16 14:13:08+00:00 Checking the status of inference endpoint wine-default-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [38]:
import json

input_payload = json.dumps({ 
    "data": [
        [14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]
    ],
    "method": "predict"  # For classification model, get probabilities with 'predict_proba'.
})

output = service.run(input_payload)

print(output)

{'predict': [0]}


In [39]:
import json

input_payload = json.dumps({ 
    "data": [
        [14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]
    ],
    "method": "predict_proba"  # For classification model, get probabilities with 'predict_proba'.
})

output = service.run(input_payload)

print(output)

{'predict_proba': [[1.0, 0.0, 0.0]]}
